# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm 
import time
import torch.optim as optim
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.features = nn.Sequential(    
            nn.Conv2d(1, 6, kernel_size=3, stride=1, padding=2),  
            nn.ReLU(inplace=True),  # inplace:直接覆盖原值，节省内存
            # Pool1:
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(True),
            # Pool2:
            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Sequential(
            # FC1
            # nn.Dropout(p=0.5),  # 随即失活，防止过拟合
            nn.Linear(16 * 5 * 5, 120),  # 相当于求Z = WX+b
            nn.ReLU(True),
            # FC2
            # nn.Dropout(p=0.5),
            nn.Linear(120, 84),
            nn.ReLU(True),
            # FC3
            nn.Linear(84, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    # def num_flat_features(self, x):
    #     size = x.size()[1:]  # all dimensions except the batch dimension
    #     num_features = 1
    #     for s in size:
    #         num_features *= s
    #     return num_features



    
model = SimpleNet().cuda()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    sys.stdout.flush()
    print("######################## EPOCH {} ##########################\n".format(epoch))
    sys.stdout.flush()
    train_correct = 0
    train_total = 0
    for images, labels in tqdm(train_loader):
        images, labels = Variable(images).cuda(), Variable(labels).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(images) # 将数据传入网络进行前向运算
        loss = criterion(outputs, labels) # 得到损失函数
        loss.backward() # 反向传播
        optimizer.step() # 通过梯度做一步参数更新

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)# labels 的长度
        train_correct += (predicted == labels).sum().item() # 预测正确的数
    sys.stdout.flush()
    print('\nFinished Training\n')
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    sys.stdout.flush()
    print("Begin Testing")
    sys.stdout.flush()
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = Variable(images).cuda(), Variable(labels).cuda()
            outputs = model(images)
            loss = criterion(outputs, labels) # 得到损失函数
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    ################计算各个分类的准确率###################
    # class_correct = list(0. for i in range(10))
    # class_total = list(0. for i in range(10))
    # with torch.no_grad():
    #     for data in test_loader:
    #         images, labels = data
    #         images, labels = Variable(images).cuda(), Variable(labels).cuda()
    #         outputs = model(images)
    #         _, predicted = torch.max(outputs, 1)
    #         c = (predicted == labels).squeeze()
    #         for i in range(4):
    #             label = labels[i]
    #             class_correct[label] += c[i].item()
    #             class_total[label] += 1
    # for i in range(10):
    #     print('Accuracy of %s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))
    sys.stdout.flush()
    print("\nFinished Testing\n")
    sys.stdout.flush()
    print("Training accuracy: {:.2f}%, Testing accuracy: {:.2f}%\n".format((100.*train_correct/train_total),(100.*test_correct/test_total)))
    
    
    


######################## EPOCH 0 ##########################

100%|██████████| 468/468 [00:13<00:00, 34.02it/s]
Finished Training

Begin Testing

100%|██████████| 78/78 [00:01<00:00, 41.23it/s]
Finished Testing

Training accuracy: 99.39%, Testing accuracy: 98.84%

######################## EPOCH 1 ##########################


100%|██████████| 468/468 [00:14<00:00, 31.72it/s]
Finished Training

Begin Testing

100%|██████████| 78/78 [00:01<00:00, 40.08it/s]
Finished Testing

Training accuracy: 99.44%, Testing accuracy: 98.83%

######################## EPOCH 2 ##########################


100%|██████████| 468/468 [00:16<00:00, 28.99it/s]
Finished Training

Begin Testing

100%|██████████| 78/78 [00:03<00:00, 22.92it/s]
Finished Testing

Training accuracy: 99.52%, Testing accuracy: 98.83%

######################## EPOCH 3 ##########################


100%|██████████| 468/468 [00:16<00:00, 28.18it/s]
Finished Training

Begin Testing

100%|██████████| 78/78 [00:02<00:00, 32.48it/s]
Finished Tes

#### Q5:
Please print the training and testing accuracy.